In [25]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
# using Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo
using Distributions

  Activating project at `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Project.toml`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Manifest.toml`


In [2]:
# include fmincon for ipopt
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [3]:
"""
x = fmincon(cost,equality_constraint,inequality_constraint,x_l,x_u,c_l,c_u,x0,params,diff_type)

This function uses IPOPT to minimize an objective function 

`cost(params, x)` 

With the following three constraints: 

`equality_constraint(params, x) = 0`
`c_l <= inequality_constraint(params, x) <= c_u` 
`x_l <= x <= x_u` 

Note that the constraint functions should return vectors. 

Problem specific parameters should be loaded into params::NamedTuple (things like 
cost weights, dynamics parameters, etc.). 

args:
    cost::Function                    - objective function to be minimzed (returns scalar)
    equality_constraint::Function     - c_eq(params, x) == 0 
    inequality_constraint::Function   - c_l <= c_ineq(params, x) <= c_u 
    x_l::Vector                       - x_l <= x <= x_u 
    x_u::Vector                       - x_l <= x <= x_u 
    c_l::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    c_u::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    x0::Vector                        - initial guess 
    params::NamedTuple                - problem parameters for use in costs/constraints 
    diff_type::Symbol                 - :auto for ForwardDiff, :finite for FiniteDiff 
    verbose::Bool                     - true for IPOPT output, false for nothing 

optional args:
    tol                               - optimality tolerance 
    c_tol                             - constraint violation tolerance 
    max_iters                         - max iterations 
    verbose                           - verbosity of IPOPT 

outputs:
    x::Vector                         - solution 

You should try and use :auto for your `diff_type` first, and only use :finite if you 
absolutely cannot get ForwardDiff to work. 

This function will run a few basic checks before sending the problem off to IPOPT to 
solve. The outputs of these checks will be reported as the following:

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

If you're getting stuck during the testing of one of the derivatives, try switching 
to FiniteDiff.jl by setting diff_type = :finite. 
""";

In [4]:
# import piano model
model = load_model("piano_with_one_shadow_hand_small/scene.xml")
data = init_data(model)

MuJoCo Data object

In [5]:
println("number of geoms: ",model.ngeom)
println("number of controls: ",model.nu)
println("number of states: ",model.nv+model.nq)
forward!(model, data)
println("Simulation timestep: ", model.opt.timestep)
println("Positions of joints: ", data.qpos)
println("Cartesion position of joints: ", data.geom_xpos)
println("Size of Cartesion position of joints: ", size(data.geom_xpos))
println("model ")
finger_geom_indices = [42,50,58,68,77]
"""
index finger:
    name:
        rh_shadow_hand//unnamed_geom_25
        rh_shadow_hand//unnamed_geom_26
    index:
        42
        43
    range:
        x:
        y:-0.6-0.5
        z:0-0.3

middle finger:
    name:
        rh_shadow_hand//unnamed_geom_33
        rh_shadow_hand//unnamed_geom_34
    index:
        50
        51
ring finger:
    name:
        rh_shadow_hand//unnamed_geom_41
        rh_shadow_hand//unnamed_geom_42
    index:
        58
        59
pinky finger:
    name:
        rh_shadow_hand//unnamed_geom_51
        rh_shadow_hand//unnamed_geom_52
    index:
        68
        69
thumb:
    name:
        rh_shadow_hand//unnamed_geom_60
        rh_shadow_hand//unnamed_geom_61
    index:
        77
        78
""";

number of geoms: 79
number of controls: 7
number of states: 46
Simulation timestep: 0.01
Positions of joints: [0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]
Cartesion position of joints: [0.0 0.0 0.0; -0.127 0.0 0.02; 0.0 -0.094 0.01125; -0.03 -0.08224999999999999 0.026000000000000002; 0.0 -0.07050000000000001 0.01125; 0.0 -0.047 0.01125; -0.03 -0.03474999999999999 0.026000000000000002; 0.0 -0.023499999999999993 0.01125; -0.03 -0.011249999999999996 0.026000000000000002; 0.0 0.0 0.01125; 0.0 0.023499999999999993 0.01125; -0.03 0.035750000000000004 0.026000000000000002; 0.0 0.047000000000000014 0.01125; -0.03 0.05925 0.026000000000000002; 0.0 0.07050000000000001 0.01125; -0.03 0.08275 0.026000000000000002; 0.0 0.094 0.01125; 0.3938251578954367 0.14997497683550026 0.13062742818321735; 0.29337550270087137 0.15000084946805123 0.12910710828129915; 0.3072543776776233 0.15000002300031276 0.1299999783510228; 0.21900000000000

## Test DIRCOL: moving index finger left right

In [6]:
tmax = 400
nx = model.nq + model.nv + model.na # State vector dimension
states = zeros(nx, tmax)
for t in 1:tmax
    states[:,t] = get_physics_state(model, data)
    step!(model, data)
end

In [7]:
function wrapped_mj_step(model, data, x, uk, forward)
    # given the current model and data. set the state and control to the model and perform a forward step
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[:] = uk[:]
        # println("hi33")
        # set state
        data.qpos .= x[1:model.nq]
        data.qvel .= x[(model.nq + 1):end]
    else
        # if using diff types, we need to convert the dual numbers to floats
        converted_uk = ForwardDiff.value.(uk)
        converted_x = ForwardDiff.value.(x)
        # set control
        data.ctrl[:] = converted_uk[:]
        # println("hi33")
        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end

    # take discrete dynamics step 
    if forward
        forward!(model,data)
    else
        step!(model, data) 
    end

    # return updated state k + 1
    # xkp1 = zeros(model.nq + model.nv) 
    # xkp1 .= get_physics_state(model, data)
    # finger_coordinates = data.geom_xpos[finger_geom_indices,:]
    
    return data.geom_xpos[finger_geom_indices[1],:], get_physics_state(model, data)
end

function robohand_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg = params.idx, params.N, params.xg
    model = params.model
    data = params.data
    Q, R, Qf = params.Q, params.R, params.Qf

    cost = 0.0
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        cost += 0.5*(xi-xg)'*Q*(xi-xg)+0.5*ui'*R*ui
    end

    # terminal cost 
    xt = Z[idx.x[N]]
    cost += 0.5*(xt-xg)'*Qf*(xt-xg)
    return cost
end
function robohand_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = init_data(model)
    step!(model, data)
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        _,  xip1_mujoco = wrapped_mj_step(model, data, xi, ui, false)

        c[idx.c[i]] = (xip1_mujoco - xip1)
    end
    # println(typeof(c))
    return c 
end

function robohand_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    con_1 = Z[idx.x[1]] - xic
    con_2 = Z[idx.x[N]] - xg

    return [con_1; con_2; robohand_dynamics_constraints(params,Z)]
end
function robohand_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    return zeros(eltype(Z), 0)
    # return inequality_constraints
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x = x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [26]:

function solve_finger_moving(;verbose=true)
    # instantiate model and data
    model = load_model("piano_with_one_shadow_hand_small/scene.xml")
    data = init_data(model)

    # reset the model and data
    reset!(model, data)
    
    # initiate time and time steps
    dt = 0.01
    tf = 0.1
    t_vec = 0:dt:tf
    N = length(t_vec)

    # LQR cost
    # Q size only 3 because we track one finger cartesian coordinates
    Q = 1*diagm(ones(model.nq + model.nv))
    R = 0.1*diagm(ones(model.nu))
    Qf = 10*diagm(ones(model.nq + model.nv))

    # indexing 
    # initialize index for state that only has the cartesian as states [x,y,z]
    idx = create_idx(model.nq + model.nv, model.nu, N)
 

    # initial and goal states
    step!(model, data)
    xic = get_physics_state(model, data)
    xic_geom = data.geom_xpos[finger_geom_indices[1],:]
    U_ref = zeros(model.nu)
    U_ref[4] = 0.664
    U_ref[5] = 1.1
    U_ref[2] = 0.21
    for t in 1:N-1
        data.ctrl .= U_ref
        step!(model, data)
    end
    # initial and goal state
    xg = get_physics_state(model, data)
    # xg = data.geom_xpos[finger_geom_indices[1],:]
    # reset the model and data
    
    # x_u_full_for_passing[1:(model.nq + model.nv + model.nu)] .= [get_physics_state(model, data); data.ctrl[:]]


    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)


    # primal bounds
    x_l = -1 * Inf * ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    for i = 1:N
        for j = 1:model.nq
            # hands
            x_l[idx.x[i][j]] = model.jnt_range[j,1]
            x_u[idx.x[i][j]] = model.jnt_range[j,2]
        end
    end
    for i = 1:(N-1)
        for j = 1:model.nu
            x_l[idx.u[i][j]] = model.actuator_ctrlrange[j,1]
            x_u[idx.u[i][j]] = model.actuator_ctrlrange[j,2]
        end
        # for j in [2,5,4]
        #     x_l[idx.u[i][j]] = 0
        #     x_u[idx.u[i][j]] = model.actuator_ctrlrange[j,2]
        # end
    end

    # inequality constraints
    # c_l = -1 * Inf * ones(3*(idx.N-1))
    # c_u = Inf * ones(3*(idx.N-1))
    c_l = zeros(0)
    c_u = zeros(0)
    # initial guess 
    z0 = randn(idx.nz)
    z0[idx.x[1]] .= xic
    for i = 2:N
        # z0[idx.x[i]] .= xic + ((xg.-xic)/(N))*(i)
        # z0[idx.x[i]] .= xic
        # z0[idx.x[i]] .= xg_physics
        for j = 1:model.nq
            z0[idx.x[i][j]] = rand(Uniform(model.jnt_range[j,1],model.jnt_range[j,2]))
        end
    end
    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    # diff_type = :auto 
    diff_type = :finite
    
    
    Z = fmincon(robohand_cost,robohand_equality_constraints,robohand_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-4, c_tol = 1e-4, max_iters = 500, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
end

solve_finger_moving (generic function with 1 method)

In [27]:
X, U, t_vec, params = solve_finger_moving(verbose=true)

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :finite (FiniteDiff.jl)---
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:   317952
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      576
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      323
                     variables with only upper bounds:        0
Total number of equality constraints.................:      552
Total number of inequality constraints.......

([[3.3743766923602456e-6, 8.92805762904543e-6, 3.3740042754784615e-6, 3.3749706899746873e-6, 8.936580766482672e-6, 3.374782810564741e-6, 8.931111083413684e-6, 3.374941373393786e-6, 3.375410143846851e-6, 2.0382326605572144e-6  …  -0.16765519406200816, -0.027464419801881572, -0.8902891473405634, -0.0052963289030622945, -4.223391310096946, 0.45935786485185287, 0.34952927792220234, 0.1768975309185164, 0.04555227576094313, 0.010944648656794485], [1.2077509403314953e-6, 3.873297758631496e-5, 1.2077812146719673e-6, 1.2076700046252276e-6, 3.874433365969981e-5, 1.2076781784471679e-6, 3.8738784873214585e-5, 1.207731517634608e-6, 1.2077598036452584e-6, 3.765056774757959e-5  …  -0.30400752353324806, -0.04610153943044556, -0.7245826167727435, 0.008379009856992579, -2.4137244220552283, -0.5975826642407034, 2.1026973896285943, 5.885200600054985, 10.03289042242929, 10.464006937865205], [8.94256207100843e-7, 8.096935110083599e-5, 8.942556472100701e-7, 8.942544967497785e-7, 8.097097507529726e-5, 8.94254

In [28]:
# instantiate model and data
# model = load_model("piano_with_one_shadow_hand_simplified/scene.xml")
model = load_model("piano_with_one_shadow_hand_small/scene.xml")
data = init_data(model)

# reset the model and data j in [2,5,4]
        #     z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        # end
dt = 0.01
tf = 0.1
t_vec = 0:dt:tf
N = length(t_vec)
states = zeros(nx, N)
ctrl_states = zeros(model.nu, N)
reset!(model, data)
step!(model, data)
states[:,1] = get_physics_state(model, data)
for t in 1:N-1
    data.ctrl .= U[t][:]
    step!(model, data)
    states[:,t+1] = get_physics_state(model, data)
end
println("end using controls:",data.geom_xpos[finger_geom_indices[1],:])
states3 = zeros(nx, N)
for t in 1:N
    states3[:,t] = X[t][:]
end

reset!(model, data)
data.ctrl .=0
data.qpos .= states3[1:model.nq,N]
data.qvel .= states3[model.nq+1:end,N]
forward!(model, data)
final_cart_fmincon = data.geom_xpos[finger_geom_indices[1],:]
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension

states2 = zeros(nx, N)
states2[:,1] = get_physics_state(model, data)
states2_coord = zeros(3,N)
states2_coord[:,1] = data.geom_xpos[finger_geom_indices[1],:]
U2 = zeros(model.nu)
U2[4] = 0.664
U2[5] = 1.1
U2[2] = 0.21
for t in 1:N-1
    data.ctrl .= U2
    step!(model, data)
    states2[:,t+1] = get_physics_state(model, data)
    states2_coord[:,t+1] = data.geom_xpos[finger_geom_indices[1],:]
end
println(data.geom_xpos[finger_geom_indices[1],:])
println(norm((final_cart_fmincon-data.geom_xpos[finger_geom_indices[1],:]),2))
for t in 1:N
    println(states2_coord[:,t])
end

end using controls:[0.01818051921520977, 0.0761514006309377, 0.06469775274650075]
[0.025395381454867045, 0.07193963679211916, 0.056461397004888794]
0.0063644555344766355
[0.0, 0.0, 0.0]
[0.0004900000000000147, 0.11699999999999999, 0.12000000000000001]
[-0.00039745522853294746, 0.11618415701496942, 0.11305629845488538]
[0.0004147145284350614, 0.11277704589079526, 0.102973328879172]
[0.0030054994381000857, 0.10707991862841891, 0.09221868253682486]
[0.006791496182904927, 0.09991124975829715, 0.08221422542634418]
[0.011162703930066919, 0.0921847080746731, 0.07363110262822647]
[0.015600942053661448, 0.08457642963920377, 0.06664445126841494]
[0.020412581984727694, 0.07794182133290084, 0.060768332182999533]
[0.023402564163286798, 0.07417287845560569, 0.058091885765530954]
[0.025395381454867045, 0.07193963679211916, 0.056461397004888794]


In [29]:
init_visualiser()
visualise!(model, data, trajectories = [states, states2])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [21]:
init_visualiser()
visualise!(model, data, trajectories = [states2])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [30]:
init_visualiser()
visualise!(model, data, trajectories = [states3])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.
